In [48]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import os
import time

In [49]:
class Mynetwork(nn.Module):
    def __init__(self,input_num=1 , out_num=1,hidden_num=128):
        super().__init__()
        self.MLP=nn.Sequential(
            nn.Linear(input_num, hidden_num),
            nn.ELU(),
            nn.Linear(hidden_num,hidden_num),
            nn.ELU(),
            nn.Linear(hidden_num,hidden_num),
            nn.ELU(),       
            nn.Linear(hidden_num,hidden_num),
            nn.ELU(),   
            nn.Linear(hidden_num,out_num),
        )
        pass
    def forward(self,x):
        return self.MLP(x)+self.MLP(-x)
    
def potential(x):
    poten=2*x**2
    return poten


In [50]:
exe_num=10
sigma=1e-1

total_time=time.time()
file_name='f1_model'

en_num=40
extend_num=10
epoch=200000
lr=0.01

h_bar=1
m=1
b_lap:float=-h_bar**2/(2*m)

# 同时对于库伦势函数, 取e=1, 4\pi\epsilon_0=1, E_n=-1/(2n^2)
dtype=torch.float32
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

La=-10
Lb =10
L=Lb-La  # domain length
N = 800   # number of interior points # 对时间成本来说几乎是平方量级
h :float= L / (N+1)
grid=torch.linspace(La,Lb,N+2,dtype=dtype,device=device)
grid=grid[1:-1].unsqueeze(-1)


diag = -2.0 / h**2 * torch.ones(N,device=device) * b_lap
off_diag = 1.0 / h**2 * torch.ones(N - 1,device=device) * b_lap


In [51]:

#######————————————————————————————————————########    
eig_loss_list=[]
pre_loss_list=[]

potential_list=[]
eig_list=[]

for execution in range(exe_num):
    
    V_diag=potential(grid)
    A = torch.diag(diag) + torch.diag(off_diag,diagonal=1) + torch.diag(off_diag, diagonal=-1)+torch.diag(V_diag.flatten())
    eigenvalues= torch.linalg.eigvalsh(A)
    real_en_0=eigenvalues[:en_num].detach()
    
    ####################
    rand_en=sigma*torch.randn(en_num,device=device,dtype=dtype)+1
    ####################
    real_en=real_en_0*rand_en
    
    
    # torch.manual_seed(seed=42) 
    os.makedirs(f'./{file_name}', exist_ok=True)
    
    model=Mynetwork().to(device=device,dtype=dtype)
    optimizer=torch.optim.Adam(model.parameters(),lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=50,threshold=1e-4)
    loss_fn=nn.L1Loss()
    
    pre_loss=loss_fn(real_en,real_en_0)
    pre_loss_list.append(pre_loss.item())

    init_time=time.time()
    for i in range(epoch):
        optimizer.zero_grad()
        V_diag=model(grid)
        A = torch.diag(diag) + torch.diag(off_diag,diagonal=1) + torch.diag(off_diag, diagonal=-1)+torch.diag(V_diag.flatten())
        eigenvalues= torch.linalg.eigvalsh(A)
        output=eigenvalues[:en_num]
        
        loss=loss_fn(output,real_en)
        loss.backward()
        optimizer.step()
        
        scheduler.step(loss)
        if optimizer.param_groups[0]["lr"] <= 1.1e-8:break
        
    final_loss=loss.item()
    final_time=time.time()-init_time
    final_epoch=i+1
    

    eig_loss_list.append(final_loss)
    
    eig_list.append(output.detach().cpu().numpy())
    potential_list.append(V_diag.flatten().detach().cpu().numpy())
    # torch.save(model.state_dict(),f'./{file_name}/model_para.pth')

    del model
    torch.cuda.empty_cache()
    print(f'Execution {execution+1} completed.')
    print(f'time: {time.time()-init_time:.2f}s , epoch: {final_epoch} , loss: {final_loss:.4f}')
    print('total time:',(time.time()-total_time)/60,' min') 

plt.plot(eig_loss_list,label='eig_loss')
plt.title(f'eig_loss')
plt.legend()
plt.savefig(f'./{file_name}/eig_loss.png')
plt.clf()


Execution 1 completed.
time: 7.58s , epoch: 773 , loss: 3.3671
total time: 0.12694895267486572  min
Execution 2 completed.
time: 32.18s , epoch: 3316 , loss: 2.2002
total time: 0.6633599758148193  min
Execution 3 completed.
time: 4.91s , epoch: 487 , loss: 2.9830
total time: 0.7452728311220805  min
Execution 4 completed.
time: 10.74s , epoch: 1064 , loss: 2.7844
total time: 0.9244416316350301  min
Execution 5 completed.
time: 17.52s , epoch: 1748 , loss: 2.5681
total time: 1.2165688872337341  min
Execution 6 completed.
time: 6.31s , epoch: 656 , loss: 3.0204
total time: 1.3218098282814026  min
Execution 7 completed.
time: 6.03s , epoch: 624 , loss: 2.3290
total time: 1.422501810391744  min
Execution 8 completed.
time: 6.28s , epoch: 657 , loss: 2.8144
total time: 1.5272607922554016  min
Execution 9 completed.
time: 39.04s , epoch: 4060 , loss: 2.9336
total time: 2.1780235807100934  min
Execution 10 completed.
time: 12.55s , epoch: 1301 , loss: 3.1055
total time: 2.3872821807861326  min

<Figure size 640x480 with 0 Axes>

In [52]:
grid=np.linspace(La,Lb,N+2)
grid=grid[1:-1]
real_poten=potential(grid)
for i in range(len(potential_list)):
    plt.plot(grid,potential_list[i],zorder=i+1,color='g',alpha=0.3)
plt.plot(grid,real_poten,label='real',zorder=i+10,color='r')
plt.title(f'potential band')
plt.legend()
plt.savefig(f'./{file_name}/potential_band.png')
plt.clf()

count=np.array([i for i in range(en_num)])
for i in range(len(eig_list)):
    plt.plot(count,eig_list[i],zorder=i+1,color='g',alpha=0.3)
plt.plot(count,real_en_0.cpu().numpy(),label='real',zorder=i+10,color='r')
plt.title(f'eigenvalue band')
plt.legend()
plt.savefig(f'./{file_name}/eig_band.png')
plt.clf()

print(' 所有执行完成 ')

 所有执行完成 


<Figure size 640x480 with 0 Axes>